In [1]:
# DirectionalMicrophone_cnn: so that it detects for every subject 
# We leave one trial out per subject, train over the rest of the trials, and test at the end for the left out. 

In [22]:
import torch
import torch.nn as nn
import torch.cuda as cuda
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import pdb
import pickle 
from torch.autograd import Variable
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from os import listdir
from os.path import isfile, join

import torch.nn.functional as F
import random 
import pandas as pd 
import scipy.io as sio
import csv

In [2]:
numSubjects = 17
dataDir = "/home/zeinab.schaefer/DMfiles/"
npyDataDir = "/home/zeinab.schaefer/codes/dataDM/"

In [3]:
# generate the name of the files 
# the form is FARAH_sub0XY_LS(1/4)_tr(1:4)
# 16 subjects with LS1 and LS4, subject 17 has only LS1
# LS1:Loud speaker to the left 
# LS4:Loud speaker to the right 
fns = []
labels = []
for subject in range(1,numSubjects+1):
    for spk in [1,4]:
        for trial in range(1,4+1):
            dt = []
            fn = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial)
            fns.append(fn)

##### generate name of the subject based on the speaker side and the subject number 

In [4]:
def read_by_name(fname):
    fpath = dataDir + fname
    dt = sio.loadmat(fpath)['y']
    dt = dt.transpose()
    return np.delete(dt, 29, axis=1)

In [5]:
def read_by_spk_tr(subject, spk, trial_number):
    # generate the name
    fn = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial_number)
    return fn 

In [6]:
def gen_nameLists(subjectNumber, speakerSide):
    ssides = [1,4]
    if speakerSide not in ssides:
        print("speaker side is 1 or 4")
        #break
    
    namelists = []
    for trial in range(1,5):
        fname = 'FARAH_'+'sub0%0.2d'%subjectNumber+'_LS'+str(speakerSide)+'_'+str(trial)
        namelists.append(fname)
    return namelists

In [7]:
#### saving the labels as csv file

In [8]:
dict_labels = dict()
dict_electrode_labels = dict()
for subj in range(1,numSubjects+1):
    
    rspk = gen_nameLists(subj, speakerSide=1)
    lspk = gen_nameLists(subj, speakerSide=4)
    
    for name in rspk:
        dict_labels[name] = '+1'
        
    for name in lspk:
        dict_labels[name] = '-1'

with open(npyDataDir + 'dict.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    for key, value in dict_labels.items():
        writer.writerow([key, value])

constructing the data loader 
we are taking all the subjects for training, but one trial out for each speaker for every subject for the purpose of testing. 

In [9]:
def sample_for(subject, spk, trial, label):
    fname = 'FARAH_'+'sub0%0.2d'%subject+'_LS'+str(spk)+'_'+str(trial)
    return {'fname': fname, 'label': label }

train_samples = []
valid_samples = []

for subject in range(1,18):           
    for spk in [1, 4]:
        if spk == 1:
            label = 0 # Right
        else:
            label = 1 # Left
        # randomly take three trials out of four for training
        # one trial for testing
        all_trials = range(1,5)
        train_trials = random.sample(all_trials, 3)
        valid_trials = np.setdiff1d(all_trials, train_trials)
        
        # list of validation names accordingly
        for trial in valid_trials:
            valid_samples.append(sample_for(subject, spk, trial, label))
        
        for trial in train_trials:
            train_samples.append(sample_for(subject, spk, trial, label))

In [10]:
class DirectionalMicrophoneDataset(Dataset):
    def __init__(self,samples):
        # path: the path to the npy files 
        # subjects: the list of the subjects to be loaded    
        self.path = npyDataDir 
        self.samples = samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, indx):
        itempath = self.path + self.samples[indx]['fname'] +'.npy'
        signal = np.load(itempath) # loads the data 
        signal_label = self.samples[indx]['label'] # the corresponding label 
        return {'signal': signal, 'label': signal_label}

In [11]:
class RandomWindow(object):
    def __init__(self, window_size):
        self.window_size = window_size
        
    def __call__(self, sample):
        signal, label = sample['signal'], sample['label']
        
        # generate a random interval 
        random_index = np.random.randint(signal.shape[0] - self.window_size)
        dt = signal[random_index:random_index+self.window_size,:]
        return {'signal': dt, 'label': label}

In [12]:
class ToTensor(object):
    def __call__(self, sample):
        signal, label = sample['signal'], sample['label']
        dt = torch.from_numpy(signal.transpose())
        return {'signal': dt, 'label': label}

In [13]:
class TransformedDataset(Dataset):
    def __init__(self, dataset, *transforms):
        self.dataset = dataset
        self.transforms = transforms
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        sample = self.dataset[idx]
        for transform in self.transforms:
            sample = transform(sample)
        return sample

### Constructing the validation and training data loader 

In [21]:
dmd = DirectionalMicrophoneDataset(train_samples)

In [22]:
window = RandomWindow(1000)
to_tensor = ToTensor()
dataset_train = TransformedDataset(dmd, window, to_tensor)
train_dl = DataLoader(dataset_train, batch_size=32, shuffle=True)

In [23]:
dmd_valid = DirectionalMicrophoneDataset(valid_samples)
valid_dataset = TransformedDataset(dmd_valid, window, to_tensor)
valid_dl = DataLoader(valid_dataset, batch_size=32)

### Constructing the Network Structure 

In [17]:
def conv1d(in_channels, out_channels, kernel_size, pool_size=1):
    return nn.Sequential(
        nn.Conv1d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size),
        nn.BatchNorm1d(out_channels),
        nn.MaxPool1d(pool_size),
        nn.ReLU()
    )

class DMNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.convs = nn.Sequential(
            conv1d(127, 64, kernel_size=32, pool_size=4),
            conv1d(64, 34, kernel_size=16),
            conv1d(34, 16, kernel_size=8)
        )
        
        self.pool = nn.AdaptiveAvgPool1d(8)
        self.fc1 = nn.Linear(16*8, 100)
        self.fc2 = nn.Linear(100, 2)

    def forward(self, x):
        x = self.pool(self.convs(x))
        x = x.view(x.size(0), -1)
        
        # Fully connected layer 1
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, training=True)
        
        # Fully connected layer 2
        x = self.fc2(x)
        
        return x

In [24]:
# The model
net = DMNetwork()

if cuda.is_available():
    print("cuda available")
    net = net.cuda()
else:
    net = net.cpu()

# Our loss function
criterion = nn.CrossEntropyLoss()

# Our optimizer
learning_rate = 0.0001
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate) 

cuda available


Training and Validation set 

In [ ]:
num_epochs = 2000

train_loss = []
valid_loss = []
train_accuracy = []
valid_accuracy = []

for epoch in range(num_epochs):
    ############################
    # Train
    ############################    
    iter_loss = 0.0
    correct = 0
    iterations = 0
    episodes = 10
    
    net.train()                   # Put the network into training mode
    for idx in range(episodes):
        for samples in train_dl:
            # Convert torch tensor to Variable
            items = Variable(samples['signal'])
            classes = Variable(samples['label'])
            
            # If we have GPU, shift the data to GPU
            if cuda.is_available():
                items = items.cuda()
                classes = classes.cuda()
            else:
                items = items.cpu()
                classes = classes.cpu()
            
            optimizer.zero_grad()     # Clear off the gradients from any past operation
            outputs = net(items.float())      # Do the forward pass
            loss = criterion(outputs, classes) # Calculate the loss
            #pdb.set_trace()
            #iter_loss += loss.data[0] # Accumulate the loss
            iter_loss += loss.data.item()
            loss.backward()           # Calculate the gradients with help of back propagation
            optimizer.step()          # Ask the optimizer to adjust the parameters based on the gradients
            
            # Record the correct predictions for training data 
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == classes.data).sum()
            iterations += 1
    
    # Record the training loss
    train_loss.append(iter_loss/iterations)
    # Record the training accuracy
    #train_accuracy.append((100 * correct / len(mnist_train_loader.dataset)))
    #print("Tr Error: %.2f" % (iter_loss/iterations)+ "---- Accuracy: %.2f%%" % (100/episodes * correct.item() / len(train_dl.dataset)))
    print ('Train Error: %.2f,----Acc: %.4f' %((iter_loss/iterations), (100/episodes * correct.item() / len(train_dl.dataset)))

    
    #print("Accuracy: %.2f%%" % (100 * correct / len(train_dl.dataset)))
   

    # computing the validation loss 
    loss = 0.0
    correct = 0
    iterations = 0
    
    # putting the network into evaluation mode 
    net.eval()
    
    for idx in range(10):
        for validSample in valid_dl:
            
            items = Variable(validSample['signal'])
            classes = Variable(validSample['label'])
            
            # If we have GPU, shift the data to GPU
            if cuda.is_available():
                items = items.cuda()
                classes = classes.cuda()
            else:
                items = items.cpu()
                classes = classes.cpu()
            
            outputs = net(items.float())
            loss += criterion(outputs, classes).data.item()
            #loss += criterion(outputs, classes).data[0]
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == classes.data).sum()
            iterations  +=1 
            
            
    valid_loss.append(loss/iterations)
    # Record the validation accuracy
    #pdb.set_trace()
    valid_accuracy.append(correct.item() / len(valid_dl.dataset) * 100.0 / episodes)
    #pdb.set_trace()
    print ('Val Error: %.2f,----Acc: %.4f' %(valid_loss[-1], valid_accuracy[-1]))
        

Error: 0.86---- Accuracy: 77.45%
Val Loss: 1.1815,---- Val Acc: 52.0588
Error: 0.70---- Accuracy: 77.55%
Val Loss: 0.9242,---- Val Acc: 56.7647
Error: 0.65---- Accuracy: 75.78%
Val Loss: 1.0955,---- Val Acc: 54.4118
Error: 0.53---- Accuracy: 77.16%
Val Loss: 0.7919,---- Val Acc: 56.1765
Error: 0.47---- Accuracy: 81.08%
Val Loss: 1.0289,---- Val Acc: 53.8235
Error: 0.50---- Accuracy: 79.51%
Val Loss: 0.8361,---- Val Acc: 54.4118
Error: 0.49---- Accuracy: 78.73%
Val Loss: 0.7362,---- Val Acc: 56.1765
Error: 0.48---- Accuracy: 80.00%
Val Loss: 0.7246,---- Val Acc: 52.6471
Error: 0.45---- Accuracy: 80.10%
Val Loss: 0.7740,---- Val Acc: 54.1176
Error: 0.42---- Accuracy: 79.90%
Val Loss: 0.8791,---- Val Acc: 51.1765
Error: 0.48---- Accuracy: 80.49%
Val Loss: 0.6631,---- Val Acc: 54.7059
Error: 0.40---- Accuracy: 83.33%
Val Loss: 0.9495,---- Val Acc: 57.6471
Error: 0.45---- Accuracy: 79.90%
Val Loss: 0.8863,---- Val Acc: 52.0588
Error: 0.42---- Accuracy: 81.57%
Val Loss: 0.9752,---- Val Acc: 

In [20]:
torch.save(net.state_dict(), "cnn_third_model")

In [26]:
net.load_state_dict(torch.load("cnn_third_model"))